In [ ]:
import pandas as pd
import os

# === Load Data ===
# Path to the file with per-frame dice & iou values

# Path to the file with patient IDs and diagnosis
diagnosis_file = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\scd_patientdata_xlsx.xlsx"

# edge detection
metrics_file = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\results\edge_detection_segmentation_results.xlsx"
output_file_name = "edge_segmentation_summary_by_diagnosis"

# watershed
# metrics_file = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\results\watershed_segmentation_results.xlsx"
# output_file_name = "watershed_segmentation_summary_by_diagnosis"

# # # Otsu
# metrics_file = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\results\otsu_segmentation_results.xlsx"
# output_file_name = "otsu_segmentation_summary_by_diagnosis"

# Load Excel files
metrics_df = pd.read_excel(metrics_file)
diagnosis_df = pd.read_excel(diagnosis_file)

# List of patient IDs to remove
exclude_patient_ids = ["SCD0002601", "SCD0002801", "SCD0003901"]

# Filter out the rows with excluded patient IDs in both DataFrames
metrics_df = metrics_df[~metrics_df['patient_id'].isin(exclude_patient_ids)]
diagnosis_df = diagnosis_df[~diagnosis_df['patient_id'].isin(exclude_patient_ids)]

# === Merge Data on 'patient_id' ===
merged_df = pd.merge(metrics_df, diagnosis_df, on="patient_id")

# === Filter out black masks (where dice or iou <= 0.001) ===
segmentation_exists_df = merged_df[(merged_df["dice"] > 0.001) & (merged_df["iou"] > 0.001)]

# === Function to compute mean (std) formatting as percentage with 1 decimal ===
def mean_std_format(series):
    return f"{(series.mean() * 100):.1f} " if not series.empty else "N/A"

# === Group by pathology and compute stats ===
summary_data = []
for pathology, group in merged_df.groupby("pathology"):
    dice_all = mean_std_format(group["dice"])
    iou_all = mean_std_format(group["iou"])
    
    # Filter for actual segmentation (dice/IOU > 0.001)
    filtered_group = group[(group["dice"] > 0.001) & (group["iou"] > 0.001)]
    dice_filtered = mean_std_format(filtered_group["dice"])
    iou_filtered = mean_std_format(filtered_group["iou"])
    
    summary_data.append([pathology, dice_all, iou_all, dice_filtered, iou_filtered])

# === Create summary table ===
summary_df = pd.DataFrame(
    summary_data, 
    columns=["Diagnosis", "Dice (All)", "IOU (All)", "Dice (Filtered)", "IOU (Filtered)"]
)

# === Save and display ===
# output_path = os.path.join(os.path.dirname(metrics_file), f"{output_file_name}.xlsx")
# summary_df.to_excel(output_path, index=False)

# print(f"Summary table saved to:\n{output_path}")
print(summary_df)


In [ ]:
import pandas as pd
import os

# File paths
base_dir = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\results"
files = {
    "Edge Detection": os.path.join(base_dir, "edge_segmentation_summary_by_diagnosis.xlsx"),

    "Watershed": os.path.join(base_dir, "watershed_segmentation_summary_by_diagnosis.xlsx"),
    
    "Otsu": os.path.join(base_dir, "otsu_segmentation_summary_by_diagnosis.xlsx"),
    
}

# Read and rename columns for each method
dfs = []
for method, path in files.items():
    df = pd.read_excel(path)
    df = df.rename(columns={
        "Dice (All)": f"Dice (All) - {method}",
        "IOU (All)": f"IOU (All) - {method}",
        "Dice (Filtered)": f"Dice (Filtered) - {method}",
        "IOU (Filtered)": f"IOU (Filtered) - {method}",
    })
    dfs.append(df)

# Merge all DataFrames on 'Pathology'
merged = dfs[0]
for df in dfs[1:]:
    merged = pd.merge(merged, df, on="Diagnosis", how="outer")

# Save the combined summary
output_path = os.path.join(base_dir, "segmentation_comparison_summary.xlsx")
merged.to_excel(output_path, index=False)

print(f"✅ Comparison table saved to:\n{output_path}")


In [ ]:
!pip install seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Read the merged comparison summary
file_path = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\results\segmentation_comparison_summary.xlsx"
df = pd.read_excel(file_path)

# Melt the DataFrame to long format for easier plotting
df_melted = df.melt(id_vars="Diagnosis", var_name="Metric_Technique", value_name="Score")
df_melted[['Metric', 'Technique']] = df_melted['Metric_Technique'].str.extract(r'(.+)\s*-\s*(.+)')

# Extract numeric score (mean only) and convert to percentage scale
df_melted['Score_numeric'] = df_melted['Score'].str.extract(r'([0-9.]+)').astype(float)

# Ensure Diagnosis order is consistent
df_melted["Diagnosis"] = pd.Categorical(df_melted["Diagnosis"], categories=sorted(df["Diagnosis"].unique()), ordered=True)

# Use pastel color palette
sns.set(style="whitegrid")
sns.set_palette("pastel")

# Directory to save plots
plot_dir = os.path.dirname(file_path)
os.makedirs(plot_dir, exist_ok=True)

# Plot and save one figure per metric
for metric in df_melted["Metric"].unique():
    # Remove "(All)" and "(Filtered)" from the metric label
    metric_label = metric.replace("[All]", "").replace("[Filtered]", "").strip()

    metric_data = df_melted[df_melted["Metric"] == metric]

    plt.figure(figsize=(8, 4))
    ax = sns.barplot(
        data=metric_data,
        x="Diagnosis", y="Score_numeric", hue="Technique"
    )
    # Move the legend outside the plot area
    ax.legend(title="Technique", loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)


    ax.set_title(f"Comparison by Diagnosis Using {metric}")
    ax.set_xlabel("Diagnosis")
    ax.set_ylabel(f"{metric_label} (%)")
    ax.set_ylim(0, 100)
    plt.xticks(rotation=15, ha='right')
    plt.tight_layout()

    

    # Save figure
    save_path = os.path.join(plot_dir, f"{metric.lower().replace(' ', '_')}_comparison_plot.png")
    plt.savefig(save_path, dpi=300, transparent=True)
    plt.close()

    print(f"📈 Saved: {save_path}")
